In [1]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)

In [2]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result


In [15]:
import pymongo

client = pymongo.MongoClient()
db = getattr(client, 'filtered_posts')
coll = db.posts
print(db.list_collection_names())

['new_posts']


In [16]:
posts = db.new_posts
curs= posts.find()

In [17]:
stemmer = SnowballStemmer('english')

In [18]:
lis = []
for i, doc in enumerate(curs):
#     print(doc['content'])
    lis.append(doc['content'])
    #######  SAFETY FOR DEVELOPMENT #######
#     if i > 4:
#         break;
    #######################################
    
    

In [19]:
lis

['백퍼센트 Teaser Photo II 20180726',
 'New Message Pluck\xa0Sound[ad_1]https://s3.envato.com/files/13409089/preview.mp3\nShort melodic sound with nice delayed pluck-bell. Perfect for all sorts of notifications: e-mail, SMS , ringtone, alarm clock alert, some event, etc.\n    Need personal sounds or music for your game, video, web or other purpose? Please contact me: sungamesounds@gmail.com\nPlease check other great interface sounds in my collection:\n[ad_2]SourceView On WordPress',
 '$3 Fries Vs. $100 Fries Líder Toni\xa043996647900Olhá esse video que encontrei no youtube:\nsourceView On WordPress',
 'Since I have to practice itI putting 50% off my couple commisionshttps://lauka2.deviantart.com/journal/Couple-Commisions-50-off-741423216',
 'Ich denke, man kann sagen, dass der größte Teil der\nGesellschaft Homosexualität anerkannt hat. Was man über Bisexualität jedoch\nnicht sagen kann. In unserer Gesellschaft muss man sich festlegen. Schwarz oder\nweiß, Mann oder Frau und exakt welche Rel

In [24]:
processed_docs = map(preprocess, lis)
processed_docs=list(processed_docs)

In [25]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 photo
1 teaser
2 백퍼센트
3 alarm
4 alert
5 bell
6 check
7 clock
8 collect
9 contact
10 delay


In [26]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)


In [27]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [30]:
bow_doc_4310 = bow_corpus[4310]
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                               dictionary[bow_doc_4310[i][0]], 
bow_doc_4310[i][1]))

Word 149 ("week") appears 1 time.
Word 195 ("beauti") appears 2 time.
Word 286 ("deliv") appears 1 time.
Word 389 ("guarante") appears 1 time.
Word 1011 ("kind") appears 1 time.
Word 1590 ("deliveri") appears 1 time.
Word 2551 ("everyday") appears 1 time.
Word 2598 ("vanilla") appears 1 time.
Word 2847 ("satisfact") appears 1 time.
Word 3804 ("lago") appears 1 time.
Word 4120 ("cake") appears 1 time.
Word 8398 ("thecakeplug") appears 1 time.
Word 8400 ("smcakesfactori") appears 1 time.
Word 8401 ("vpruqe") appears 1 time.


In [43]:
from gensim import corpora, models
from pprint import pprint


tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5620061629774619), (1, 0.8271330441805299)]


In [31]:
lis[4310]

'Beautiful something!\nVanilla cake ....8500\nDelivery fee 1k\nWe deliver within Lagos everyday of the week!\n#TheCakePlug @SMCakesFactory\n08132453673\nWith us your Satisfaction is guaranteed!\nKindly RT this beauty to your TL https://t.co/VPruqE9wZc'

In [41]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=40, id2word=dictionary, passes=2, chunksize=1000, workers=2)

In [42]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.028*"home" + 0.026*"hous" + 0.024*"area" + 0.023*"view" + 0.022*"beauti" + 0.018*"properti" + 0.018*"shop" + 0.017*"look" + 0.016*"visit" + 0.016*"sale"
Topic: 1 
Words: 0.063*"miss" + 0.040*"ticket" + 0.035*"see" + 0.027*"night" + 0.021*"open" + 0.021*"monday" + 0.019*"go" + 0.019*"friday" + 0.018*"juli" + 0.018*"week"
Topic: 2 
Words: 0.031*"driver" + 0.020*"cash" + 0.019*"wide" + 0.018*"replac" + 0.018*"warranti" + 0.016*"water" + 0.016*"includ" + 0.014*"move" + 0.014*"condit" + 0.014*"electr"
Topic: 3 
Words: 0.051*"twice" + 0.048*"make" + 0.036*"month" + 0.035*"break" + 0.023*"expir" + 0.023*"accept" + 0.017*"stop" + 0.017*"mark" + 0.017*"win" + 0.016*"favorit"
Topic: 4 
Words: 0.064*"urgent" + 0.056*"hospit" + 0.050*"blood" + 0.050*"need" + 0.034*"bloodmatt" + 0.033*"dialysi" + 0.028*"unit" + 0.018*"requir" + 0.017*"aug_" + 0.015*"hyderabad"
Topic: 5 
Words: 0.049*"book" + 0.037*"free" + 0.036*"august" + 0.028*"today" + 0.028*"join" + 0.023*"week" + 0.019*"cont

In [47]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=100, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

/home/kali/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:1023: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


Topic: 0 Word: 0.056*"brother" + 0.025*"kick" + 0.025*"varsiti" + 0.025*"battl" + 0.025*"abandon" + 0.023*"brett" + 0.023*"vacanc" + 0.021*"school" + 0.019*"backup" + 0.018*"digit"
Topic: 1 Word: 0.028*"cancel" + 0.026*"refund" + 0.016*"flight" + 0.013*"order" + 0.013*"lifelin" + 0.012*"passbi" + 0.012*"movi" + 0.011*"prevent" + 0.011*"collar" + 0.010*"recent"
Topic: 2 Word: 0.031*"advanc" + 0.029*"system" + 0.021*"abeg" + 0.018*"screen" + 0.018*"nowplay" + 0.015*"monitor" + 0.015*"unabl" + 0.015*"eye" + 0.015*"king" + 0.013*"asap"
Topic: 3 Word: 0.033*"comment" + 0.022*"moment" + 0.022*"instant" + 0.022*"address" + 0.022*"miracl" + 0.021*"programm" + 0.021*"carri" + 0.020*"capc" + 0.020*"download" + 0.018*"list"
Topic: 4 Word: 0.017*"lot" + 0.015*"babi" + 0.012*"android" + 0.011*"paytm" + 0.011*"doubl" + 0.010*"mysteri" + 0.010*"thur" + 0.010*"languag" + 0.009*"exhibit" + 0.009*"phone"
Topic: 5 Word: 0.158*"articl" + 0.151*"bread" + 0.044*"massag" + 0.031*"use" + 0.030*"broadcast" + 0